# Model Preparation

This notebook is dedicated at exploring some models, number of parameters, complexities, etc.

In [1]:
import numpy as np
import os
import sys
import torch
import torch.nn as nn
from torch.nn import functional as F
import thop


In [2]:
from model_blocks import *
from models import *
from utils import *
from tools import *

In [3]:
from apex import amp

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [5]:
codebook_name = "codes/adhoc-codebook-1871.pkl"
import pickle
codebook_path = '/home/leo/projects/mix_nlp/utf8/codes/adhoc-codebook-1871.pkl'
f = open(codebook_path, 'rb')
codebook, char2int, int2char = pickle.load(f)

In [6]:
dynmodel = DynConvColModel(codebook)

/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux

In [7]:
dynmodel = dynmodel.to(device)

In [8]:
count_parameters(dynmodel), count_trainable_parameters(dynmodel)

(18191306, 18011690)

In [9]:
%%time 
invect = torch.randint(0,2**10, (24, 512)).to(device)

CPU times: user 0 ns, sys: 5.57 ms, total: 5.57 ms
Wall time: 8.07 ms


In [10]:
r = dynmodel(invect)

NotImplementedError: 

In [9]:
macs, flops = thop.profile(dynmodel, inputs=(invect,))
macs, flops

NotImplementedError: 

In [11]:
import torch
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.utils import weight_norm
# from fairseq.modules.dynamic_convolution import DynamicConv
from fairseq.models.lightconv import LightConvEncoderLayer, base_architecture
from fairseq.modules.positional_embedding import PositionalEmbedding
seq_len=512
lang_seq_len=60
vocab_size=1871
in_dim=96
hidd_dim=1024
emb_dim=192
enc_dropout=0.1
encoder_kernel_size_list=[3, 7, 15, 31, 31, 31, 31]
encoder_glu=True
weight_softmax=True
encoder_attention_heads=8
encoder_ff_size=2048


embed_scale = math.sqrt(emb_dim)
args = argparse.Namespace()
args.dropout = 0.1
args.encoder_conv_type = "dynamic"
args.encoder_kernel_size_list = encoder_kernel_size_list
args.encoder_glu = encoder_glu
args.weight_softmax = weight_softmax
args.encoder_attention_heads = encoder_attention_heads
args.weight_dropout = enc_dropout
args.relu_dropout = 0.
args.input_dropout = 0.
args.encoder_normalize_before = False
# args.encoder_normalize_after = True
args.encoder_layers = len(encoder_kernel_size_list)
args.encoder_ffn_embed_dim = encoder_ff_size
args.encoder_embed_dim = emb_dim
args.encoder_conv_dim = args.encoder_embed_dim
args.max_source_positions = 4096  # , 8192, 16384
base_architecture(args)

embed_positions = PositionalEmbedding(args.max_source_positions, emb_dim, padding_idx=0)

layers = nn.ModuleList([])
layers.extend([
    LightConvEncoderLayer(args, kernel_size=args.encoder_kernel_size_list[i])
    for i in range(args.encoder_layers)
])


/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_32E
/home/leo/venv3/lib/python3.7/site-packages/dynamicconv_layer-0.0.0-py3.7-linux-x86_64.egg/dynamicconv_cuda.cpython-37m-x86_64-linux

ModuleList(
  (0): LightConvEncoderLayer(
    dropout=0.1, relu_dropout=0.0, input_dropout=0.0, normalize_before=False
    (linear1): Linear(in_features=192, out_features=384, bias=True)
    (act): GLU(dim=-1)
    (conv): DynamicConv1dTBC(
      192, kernel_size=3, padding_l=1, num_heads=8, weight_softmax=True, conv_bias=False, renorm_padding=False, in_proj=False, weight_dropout=0.1
      (weight_linear): Linear(in_features=192, out_features=24, bias=False)
    )
    (linear2): Linear(in_features=192, out_features=192, bias=True)
    (fc1): Linear(in_features=192, out_features=2048, bias=True)
    (fc2): Linear(in_features=2048, out_features=192, bias=True)
    (layer_norms): ModuleList(
      (0): FusedLayerNorm(torch.Size([192]), eps=1e-05, elementwise_affine=True)
      (1): FusedLayerNorm(torch.Size([192]), eps=1e-05, elementwise_affine=True)
    )
  )
  (1): LightConvEncoderLayer(
    dropout=0.1, relu_dropout=0.0, input_dropout=0.0, normalize_before=False
    (linear1): Linear(in

In [12]:
invect = torch.rand(size=(1, 384, 192)).to(device)
type(invect)

torch.Tensor

In [13]:
%%time
layers.to(device)
r = layers[0](invect,encoder_padding_mask=None)

CPU times: user 2.05 s, sys: 765 ms, total: 2.82 s
Wall time: 2.82 s


In [5]:
mixedblock = MixtureEncoderBlock().to(device)

In [6]:
count_parameters(mixedblock), count_trainable_parameters(mixedblock)

(3467520, 3467520)

In [7]:
count_parameters(mixedblock), count_trainable_parameters(mixedblock)

(3467520, 3467520)

In [8]:
# invect = torch.randint(0,2**10, (1024, 2048))
# invect = torch.randn((240, 2048, 192)).to(device) # 240 as batch size is the MAX it supports there in my memory... 
invect = torch.randn((24, 2048, 256)).to(device) 

In [9]:
# %time macs, flops = thop.profile(mixedblock, inputs=(invect,))

In [10]:
# macs, flops

In [11]:
# mixedblock(invect)

In [12]:
# macs, flops

In [13]:
cm = ConvModel(codebook).to(device)

In [14]:
# cm.embeds.requires_grad=False

In [15]:
count_parameters(cm), count_trainable_parameters(cm)

(14731282, 14551666)

In [16]:
# cm.embeds.requires_grad_(False)

In [17]:
%%time 
invect = torch.randint(0,2**10, (24, 512)).to(device)
macs, flops = thop.profile(cm, inputs=(invect,))
macs, flops

Either .total_ops or .total_params is already defined in Conv1d(512, 1024, kernel_size=(1,), stride=(1,)). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Dropout(p=0.3, inplace=False). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(1,), stride=(1,)). Be careful, it might change your code's behavior.
Ei

CPU times: user 58 ms, sys: 24.5 ms, total: 82.5 ms
Wall time: 78.6 ms


/home/leo/venv3/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(239024029696.0, 13904398.0)

In [18]:
# cm(invect)

In [ ]:
# checkpoint = {
#                 'model': model.state_dict(),
#                 'optimizer': optimizer.state_dict(),
#                 'amp': amp.state_dict()
#             }
import os
chkp_path = "/media/nfs/mix_nlp/checkpoints"
chkp_fname = os.path.join(chkp_path, "amp-checkpoint_opt-O2_loss-0.001_2020-02-15T11:12:26.762745.pt")
# chkp_fname = os.path.join(chkp_path, "")
# chkp_fname = os.path.join(chkp_path, "")

def load_checkpoint(clean_model, fname, optimizer=None, amp=None):
    chkp = torch.load(fname)
    clean_model.load_state_dict(chkp['model'])
    if optimizer:
        optimizer.load_state_dict(chkp['optimizer'])
    if amp:
        amp.load_state_dict(chkp['amp'])
    return chkp



AttributeError: cannot assign module before Module.__init__() call